<a href="https://colab.research.google.com/github/abhinav9629/Yoga107/blob/main/DenseNet_POSE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [34]:
dataset_path = '/content/drive/MyDrive/DATASETS/YogaDataset/poseDataset'

In [35]:
labels = []
for label in os.listdir(dataset_path):
  labels.append(label)
print(len(labels))

107


In [36]:
train_set=[]
test_set=[]
count=0
for files in os.listdir(dataset_path):
    t=0
    path=os.path.join(dataset_path,files)
    for img in os.listdir(path):
        image=load_img(os.path.join(path,img), grayscale=False, color_mode='rgb', target_size=(128,128))
        image=img_to_array(image)
        image=image/255.0
        if t<60:
            train_set.append([image,count])
        else:
            test_set.append([image,count])
        t=t+1
    count=count+1
    print(count,end=" ")

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 

In [37]:
train,labels0 = zip(*train_set)
test,testlabels0 = zip(*test_set)

In [38]:
labels1 = to_categorical(labels0)
label = np.array(labels1) 

In [39]:
train = np.array(train)
test = np.array(test)

In [40]:
trainx,testx,trainy,testy=train_test_split(train,label,test_size=0.2,random_state=44)

In [41]:
datagen = ImageDataGenerator(horizontal_flip=True,vertical_flip=True,rotation_range=20,zoom_range=0.2,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.1,fill_mode="nearest")

In [ ]:
#pretrained_model = tf.keras.applications.DenseNet201(input_shape=(40,40,3),include_top=False,weights='imagenet',pooling='avg')
#pretrained_model.trainable = False

In [ ]:
#pretrained_model = keras.applications.ResNet101(input_shape=(40,40,3),include_top=False,weights='imagenet',pooling="avg")
#pretrained_model.trainable = False

In [ ]:
'''
inputs = pretrained_model.input
x = keras.layers.Dense(512, activation='relu')(pretrained_model.output)
x = keras.layers.Dropout(0.3)(x)
#x = keras.layers.Dense(256,activation='relu')(x)
#x = keras.layers.Dropout(0.3)(x)
outputs = tf.keras.layers.Dense(107, activation='softmax')(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)
'''

"\ninputs = pretrained_model.input\nx = keras.layers.Dense(512, activation='relu')(pretrained_model.output)\nx = keras.layers.Dropout(0.3)(x)\n#x = keras.layers.Dense(256,activation='relu')(x)\n#x = keras.layers.Dropout(0.3)(x)\noutputs = tf.keras.layers.Dense(107, activation='softmax')(x)\nmodel = tf.keras.Model(inputs=inputs, outputs=outputs)\n"

In [ ]:
#CUSTOM MODEL 1
'''
model = keras.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu',padding = 'Same', input_shape=(40, 40, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.25),
    #tf.keras.layers.Conv2D(128, (3,3), activation='relu',padding = 'Same'),
    #tf.keras.layers.MaxPooling2D(2,2),
    #tf.keras.layers.Dropout(0.25),
    #tf.keras.layers.Conv2D(128, (3,3), activation='relu',padding = 'Same'),
    #tf.keras.layers.MaxPooling2D(2,2),
    #tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu',padding = 'Same'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(256, (3,3), activation='relu',padding = 'Same'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(107, activation='softmax')
])
'''

"\nmodel = keras.Sequential([\n    tf.keras.layers.Conv2D(64, (3,3), activation='relu',padding = 'Same', input_shape=(40, 40, 3)),\n    tf.keras.layers.MaxPooling2D(2, 2),\n    tf.keras.layers.Dropout(0.25),\n    #tf.keras.layers.Conv2D(128, (3,3), activation='relu',padding = 'Same'),\n    #tf.keras.layers.MaxPooling2D(2,2),\n    #tf.keras.layers.Dropout(0.25),\n    #tf.keras.layers.Conv2D(128, (3,3), activation='relu',padding = 'Same'),\n    #tf.keras.layers.MaxPooling2D(2,2),\n    #tf.keras.layers.Dropout(0.25),\n    tf.keras.layers.Conv2D(128, (3,3), activation='relu',padding = 'Same'),\n    tf.keras.layers.MaxPooling2D(2,2),\n    tf.keras.layers.Dropout(0.25),\n    tf.keras.layers.Conv2D(256, (3,3), activation='relu',padding = 'Same'),\n    tf.keras.layers.MaxPooling2D(2,2),\n    tf.keras.layers.Dropout(0.25),\n    tf.keras.layers.Flatten(),\n    tf.keras.layers.Dense(1024, activation='relu'),\n    tf.keras.layers.Dropout(0.5),\n    tf.keras.layers.Dense(107, activation='softmax')\

In [42]:
#CUSTOM MODEL 2
model = keras.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu',padding = 'Same', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu',padding = 'Same'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu',padding = 'Same'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu',padding = 'Same'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(256, (3,3), activation='relu',padding = 'Same'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(107, activation='softmax')
])


In [48]:
optimiser = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimiser,loss='categorical_crossentropy',metrics=['accuracy'])

In [50]:
class MyThresholdCallback(tf.keras.callbacks.Callback):
    def __init__(self, threshold):
        super(MyThresholdCallback, self).__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None): 
        accuracy = logs["val_accuracy"]
        if accuracy >= self.threshold:
            self.model.stop_training = True


In [70]:
callback = MyThresholdCallback(0.9)

In [54]:
model_history = model.fit(datagen.flow(trainx,trainy,batch_size=32),validation_data=(testx,testy), callbacks=[callback], epochs = 1000)

Epoch 1/1000
134/134 [==============================] - 15s 110ms/step - loss: 4.6499 - accuracy: 0.0149 - val_loss: 4.6662 - val_accuracy: 0.0065
Epoch 2/1000
134/134 [==============================] - 15s 112ms/step - loss: 4.6506 - accuracy: 0.0096 - val_loss: 4.6625 - val_accuracy: 0.0103
Epoch 3/1000
134/134 [==============================] - 15s 110ms/step - loss: 4.6406 - accuracy: 0.0147 - val_loss: 4.6535 - val_accuracy: 0.0121
Epoch 4/1000
134/134 [==============================] - 15s 112ms/step - loss: 4.6182 - accuracy: 0.0145 - val_loss: 4.6324 - val_accuracy: 0.0205
Epoch 5/1000
134/134 [==============================] - 15s 110ms/step - loss: 4.5864 - accuracy: 0.0194 - val_loss: 4.6192 - val_accuracy: 0.0280
Epoch 6/1000
134/134 [==============================] - 15s 110ms/step - loss: 4.5573 - accuracy: 0.0229 - val_loss: 4.5867 - val_accuracy: 0.0373
Epoch 7/1000
134/134 [==============================] - 15s 110ms/step - loss: 4.5310 - accuracy: 0.0255 - val_loss: 4

In [55]:
model.evaluate(testx,testy)

34/34 [==============================] - 0s 7ms/step - loss: 1.7154 - accuracy: 0.6088


[1.715355396270752, 0.608776867389679]

In [ ]:
#PREDICTION

In [56]:
Val = []
for i in range(len(labels)):
  Val.append(i)

reverse_mapping=dict(zip(Val,labels)) 

In [68]:
image=load_img("/content/drive/MyDrive/DATASETS/YogaDataset/poseDataset/agnistambhasana/61-0.png",target_size=(128,128))

image=img_to_array(image) 
image=image/255.0
prediction_image=np.array(image)
prediction_image= np.expand_dims(image, axis=0)

In [69]:
prediction=model.predict(prediction_image)
value=np.argmax(prediction)
move_name = reverse_mapping[value]
print("Prediction is {}.".format(move_name))

Prediction is agnistambhasana.


In [ ]:
model.save('/content/drive/MyDrive/model/try.h5')